In [43]:
// Instalación de Paquetes necesarios

// #r "nuget:Microsoft.Data.Analysis,0.19.0"
#r "nuget: Microsoft.ML, 1.7.1"
#r "nuget: System.Data.SqlClient, 4.8.4"

Installed Packages Microsoft.ML, 1.7.1 System.Data.SqlClient, 4.8.4

In [45]:
#r "nuget:Microsoft.DotNet.Interactive.SqlServer,*-*"

Installed Packages Microsoft.DotNet.Interactive.SqlServer, 1.0.0-beta.22553.7

Loading extensions from `Microsoft.DotNet.Interactive.SqlServer.dll`

Query Microsoft SQL Server databases. 
 This extension adds support for connecting to Microsoft SQL Server databases using the #!connect mssql magic command. For more information, run a cell using the #!sql magic command.

In [51]:
#!connect mssql --kernel-name mydatabase "Persist Security Info=False; Integrated Security=false; Initial Catalog=AdventureWorksDW2017; Server=<SERVIDOR>; User = <USUARIO>;Password=<PWD>"

Kernel added: #!sql-mydatabase

In [54]:
#!sql-mydatabase
SELECT TOP 5 * FROM dbo.clientes

(5 filas afectadas)

CustomerKey,rfm_recency,rfm_frequency,rfm_monetary,Edad,EstadoCivil,Sexo,Ingresos,Hijos,HijosEnCasa,NivelEducativo,Ocupacion,CasaEnPropiedad,NumeroCoches,MesesPrimeraCompra,DistanciaAlTrabajo
14324,2,4,2,52,S,F,"70000,00",1,0,Estudios universitarios (en curso),Obrero especializado,1,1,17,5-10 Miles
18569,1,1,1,43,M,F,"60000,00",0,0,Estudios de postgrado,Profesional,1,0,12,0-1 Miles
27059,1,4,1,31,M,M,"50000,00",0,0,Estudios de postgrado,Obrero especializado,0,0,12,0-1 Miles
22814,1,1,1,63,M,M,"100000,00",2,3,Licenciatura,Gestión,1,4,11,10+ Miles
15652,2,4,2,48,M,F,"30000,00",4,0,Estudios de postgrado,Administrativo,1,0,27,0-1 Miles


In [26]:
using System;
using System.IO;
using System.Data.SqlClient;

//using Microsoft.Data.Analysis;

using Microsoft.ML;
using Microsoft.ML.Data;
using Microsoft.ML.Transforms;


// using CustomerSegmentation.DataStructures;

In [27]:
// Definimos la clase para los datos a cargar. Es necesaria para cargar los datos desde BBDD en MLContext
public class CustomerData
{
    public float CustomerKey { get; set; }
    public float NumeroCoches { get; set; }
    public float rfm_recency {get;set;}
    public float rfm_frequency {get;set;}
    public float rfm_monetary {get;set;}
    public float Edad {get;set;}
    public string EstadoCivil {get;set;}
    public string Sexo {get;set;}
    public float Hijos {get;set;}
    public float HijosEnCasa {get;set;}
    public string NivelEducativo {get;set;}
    public string Ocupacion {get;set;} 
    public float MesesPrimeraCompra {get;set;} 
    public string DistanciaAlTrabajo {get;set;}
}

In [28]:
// Definimos el contexto y el esquema de datos a cargar
MLContext mlContext = new MLContext();
DatabaseLoader loader = mlContext.Data.CreateDatabaseLoader<CustomerData>();

// Cargamos los datos
string connectionString = @"Server=tcp:<SERVIDOR>,1433;Database=AdventureWorksDW2017;User ID=<USUARIO>;Password=<PWD>;Trusted_Connection=False;Encrypt=True;";
string sqlCommand = @"SELECT CAST(CustomerKey as real) as CustomerKey, CAST(NumeroCoches as real) as NumeroCoches, CAST(rfm_recency as real) as rfm_recency, CAST(rfm_frequency as real) as rfm_frequency, 
    CAST(rfm_monetary as real) as rfm_monetary, CAST(Edad as real) as Edad,EstadoCivil,Sexo, CAST(Hijos as real) as Hijos, CAST(HijosEnCasa as real) as HijosEnCasa, 
    NivelEducativo, Ocupacion, CAST(MesesPrimeraCompra as real) as MesesPrimeraCompra, DistanciaAlTrabajo
FROM [dbo].[clientes]";

//Creamos el objeto para conectar al origen y ejecutar la consulta
DatabaseSource dbSource = new DatabaseSource(SqlClientFactory.Instance, connectionString, sqlCommand);
// Cargamos los datos en el DataView
IDataView data = loader.Load(dbSource);

In [29]:
// Como ejemplo, no tiene sentido en un problema de cluster
var trainTestData = mlContext.Data.TrainTestSplit(data, testFraction: 0.2);
var trainingDataView = trainTestData.TrainSet;
var testingDataView = trainTestData.TestSet;

In [30]:
data.Preview(10)

Schema,ColumnView,RowView
"[ { CustomerKey: Single: Name: CustomerKey, Index: 0, IsHidden: False, Type: { Single: RawType: System.Single }, Annotations: { : Schema: [ ] } }, { NumeroCoches: Single: Name: NumeroCoches, Index: 1, IsHidden: False, Type: { Single: RawType: System.Single }, Annotations: { : Schema: [ ] } }, { rfm_recency: Single: Name: rfm_recency, Index: 2, IsHidden: False, Type: { Single: RawType: System.Single }, Annotations: { : Schema: [ ] } }, { rfm_frequency: Single: Name: rfm_frequency, Index: 3, IsHidden: False, Type: { Single: RawType: System.Single }, Annotations: { : Schema: [ ] } }, { rfm_monetary: Single: Name: rfm_monetary, Index: 4, IsHidden: False, Type: { Single: RawType: System.Single }, Annotations: { : Schema: [ ] } }, { Edad: Single: Name: Edad, Index: 5, IsHidden: False, Type: { Single: RawType: System.Single }, Annotations: { : Schema: [ ] } }, { EstadoCivil: String: Name: EstadoCivil, Index: 6, IsHidden: False, Type: { String: RawType: System.ReadOnlyMemory<System.Char> }, Annotations: { : Schema: [ ] } }, { Sexo: String: Name: Sexo, Index: 7, IsHidden: False, Type: { String: RawType: System.ReadOnlyMemory<System.Char> }, Annotations: { : Schema: [ ] } }, { Hijos: Single: Name: Hijos, Index: 8, IsHidden: False, Type: { Single: RawType: System.Single }, Annotations: { : Schema: [ ] } }, { HijosEnCasa: Single: Name: HijosEnCasa, Index: 9, IsHidden: False, Type: { Single: RawType: System.Single }, Annotations: { : Schema: [ ] } }, { NivelEducativo: String: Name: NivelEducativo, Index: 10, IsHidden: False, Type: { String: RawType: System.ReadOnlyMemory<System.Char> }, Annotations: { : Schema: [ ] } }, { Ocupacion: String: Name: Ocupacion, Index: 11, IsHidden: False, Type: { String: RawType: System.ReadOnlyMemory<System.Char> }, Annotations: { : Schema: [ ] } }, { MesesPrimeraCompra: Single: Name: MesesPrimeraCompra, Index: 12, IsHidden: False, Type: { Single: RawType: System.Single }, Annotations: { : Schema: [ ] } }, { DistanciaAlTrabajo: String: Name: DistanciaAlTrabajo, Index: 13, IsHidden: False, Type: { String: RawType: System.ReadOnlyMemory<System.Char> }, Annotations: { : Schema: [ ] } } ]","[ { CustomerKey: Single: Column: { CustomerKey: Single: Name: CustomerKey, Index: 0, IsHidden: False, Type: { Single: RawType: System.Single }, Annotations: { : Schema: [ ] } }, Values: [ 11000, 11001, 11002, 11003, 11004, 11005, 11006, 11007, 11008, 11009 ] }, { NumeroCoches: Single: Column: { NumeroCoches: Single: Name: NumeroCoches, Index: 1, IsHidden: False, Type: { Single: RawType: System.Single }, Annotations: { : Schema: [ ] } }, Values: [ 0, 1, 1, 1, 4, 1, 1, 2, 3, 1 ] }, { rfm_recency: Single: Column: { rfm_recency: Single: Name: rfm_recency, Index: 2, IsHidden: False, Type: { Single: RawType: System.Single }, Annotations: { : Schema: [ ] } }, Values: [ 1, 3, 1, 1, 1, 1, 1, 1, 1, 1 ] }, { rfm_frequency: Single: Column: { rfm_frequency: Single: Name: rfm_frequency, Index: 3, IsHidden: False, Type: { Single: RawType: System.Single }, Annotations: { : Schema: [ ] } }, Values: [ 5, 5, 3, 5, 4, 4, 4, 5, 4, 4 ] }, { rfm_monetary: Single: Column: { rfm_monetary: Single: Name: rfm_monetary, Index: 4, IsHidden: False, Type: { Single: RawType: System.Single }, Annotations: { : Schema: [ ] } }, Values: [ 2, 2, 2, 2, 2, 2, 2, 2, 2, 2 ] }, { Edad: Single: Column: { Edad: Single: Name: Edad, Index: 5, IsHidden: False, Type: { Single: RawType: System.Single }, Annotations: { : Schema: [ ] } }, Values: [ 43, 38, 43, 41, 35, 38, 38, 45, 39, 45 ] }, { EstadoCivil: String: Column: { EstadoCivil: String: Name: EstadoCivil, Index: 6, IsHidden: False, Type: { String: RawType: System.ReadOnlyMemory<System.Char> }, Annotations: { : Schema: [ ] } }, Values: [ M, S, M, S, S, S, S, M, S, S ] }, { Sexo: String: Column: { Sexo: String: Name: Sexo, Index: 7, IsHidden: False, Type: { String: RawType: System.ReadOnlyMemory<System.Char> }, Annotations: { : Schema: [ ] } }, Values: [ M, M, M, F, F, M, F, M, F, M ] }, { Hijos

In [31]:
public static void ShowDataViewInConsole(MLContext mlContext, IDataView dataView, int numberOfRows = 4)
{
    string msg = string.Format("Show data in DataView: Showing {0} rows with the columns", numberOfRows.ToString());
    

    var preViewTransformedData = dataView.Preview(maxRows: numberOfRows);

    foreach (var row in preViewTransformedData.RowView)
    {
        var ColumnCollection = row.Values;
        string lineToPrint = "Row--> ";
        foreach (KeyValuePair<string, object> column in ColumnCollection)
        {
            lineToPrint += $"| {column.Key}:{column.Value}";
        }
        Console.WriteLine(lineToPrint + "\n");
    }
}

In [32]:
ShowDataViewInConsole(mlContext, data)

Row--> | CustomerKey:11000| NumeroCoches:0| rfm_recency:1| rfm_frequency:5| rfm_monetary:2| Edad:43| EstadoCivil:M| Sexo:M| Hijos:2| HijosEnCasa:0| NivelEducativo:Licenciatura| Ocupacion:Profesional| MesesPrimeraCompra:37| DistanciaAlTrabajo:1-2 Miles

Row--> | CustomerKey:11001| NumeroCoches:1| rfm_recency:3| rfm_frequency:5| rfm_monetary:2| Edad:38| EstadoCivil:S| Sexo:M| Hijos:3| HijosEnCasa:3| NivelEducativo:Licenciatura| Ocupacion:Profesional| MesesPrimeraCompra:37| DistanciaAlTrabajo:0-1 Miles

Row--> | CustomerKey:11002| NumeroCoches:1| rfm_recency:1| rfm_frequency:3| rfm_monetary:2| Edad:43| EstadoCivil:M| Sexo:M| Hijos:3| HijosEnCasa:3| NivelEducativo:Licenciatura| Ocupacion:Profesional| MesesPrimeraCompra:37| DistanciaAlTrabajo:2-5 Miles

Row--> | CustomerKey:11003| NumeroCoches:1| rfm_recency:1| rfm_frequency:5| rfm_monetary:2| Edad:41| EstadoCivil:S| Sexo:F| Hijos:0| HijosEnCasa:0| NivelEducativo:Licenciatura| Ocupacion:Profesional| MesesPrimeraCompra:38| DistanciaAlTrabajo

In [34]:
// Definimos un estimador para reemplazar los valores nulos
// Imputamos nulos en la columna Edad, por ejemplo
var replacementEstimator = mlContext.Transforms.ReplaceMissingValues(new[] 
    {
        new InputOutputColumnPair("Edad"),
        new InputOutputColumnPair("rfm_recency"),
        new InputOutputColumnPair("rfm_frequency"),
        new InputOutputColumnPair("rfm_monetary"),
        new InputOutputColumnPair("Hijos"),
        new InputOutputColumnPair("HijosEnCasa"),
        new InputOutputColumnPair("NumeroCoches"),
        new InputOutputColumnPair("MesesPrimeraCompra")
    
    }, replacementMode: MissingValueReplacingEstimator.ReplacementMode.Mean);

// Fit data to estimator
// Fitting generates a transformer that applies the operations of defined by estimator
ITransformer replacementTransformer = replacementEstimator.Fit(data);

// Transform data
IDataView transformedData = replacementTransformer.Transform(data);

In [35]:
// OneHot Encoding
var multiColumnKeyPipeline = mlContext.Transforms.Categorical.OneHotEncoding(new[]
                    {
                        new InputOutputColumnPair("DistanciaAlTrabajo"),
                        new InputOutputColumnPair("NivelEducativo"),
                        new InputOutputColumnPair("EstadoCivil"),
                        new InputOutputColumnPair("Sexo"),
                        new InputOutputColumnPair("Ocupacion")
                    });

// Fit and Transform data.
IDataView transformedData2 = multiColumnKeyPipeline.Fit(transformedData).Transform(transformedData);

In [36]:
ShowDataViewInConsole(mlContext, transformedData2)

Row--> | CustomerKey:11000| NumeroCoches:0| NumeroCoches:0| rfm_recency:1| rfm_recency:1| rfm_frequency:5| rfm_frequency:5| rfm_monetary:2| rfm_monetary:2| Edad:43| Edad:43| EstadoCivil:M| EstadoCivil:1| EstadoCivil:Sparse vector of size 2, 1 explicit values| Sexo:M| Sexo:1| Sexo:Sparse vector of size 2, 1 explicit values| Hijos:2| Hijos:2| HijosEnCasa:0| HijosEnCasa:0| NivelEducativo:Licenciatura| NivelEducativo:1| NivelEducativo:Sparse vector of size 5, 1 explicit values| Ocupacion:Profesional| Ocupacion:1| Ocupacion:Sparse vector of size 5, 1 explicit values| MesesPrimeraCompra:37| MesesPrimeraCompra:37| DistanciaAlTrabajo:1-2 Miles| DistanciaAlTrabajo:1| DistanciaAlTrabajo:Sparse vector of size 5, 1 explicit values

Row--> | CustomerKey:11001| NumeroCoches:1| NumeroCoches:1| rfm_recency:3| rfm_recency:3| rfm_frequency:5| rfm_frequency:5| rfm_monetary:2| rfm_monetary:2| Edad:38| Edad:38| EstadoCivil:S| EstadoCivil:2| EstadoCivil:Sparse vector of size 2, 1 explicit values| Sexo:M| Se

In [37]:
// para crear el pipeline de entranamiento
// paso 1: concatenamos características de entrada en una única columna
var pipeline = mlContext.Transforms.Concatenate(
    "Features", 
    "NumeroCoches", 
    "rfm_recency", 
    "rfm_frequency",
    "rfm_monetary" ,
    "Edad",
    "EstadoCivil",
    "Sexo",
    "Hijos",
    "HijosEnCasa",
    "NivelEducativo",
    "Ocupacion",
    "MesesPrimeraCompra",
    "DistanciaAlTrabajo")

    // paso 2: usamos el algoritmo de k-means 
    
    .Append(mlContext.Clustering.Trainers.KMeans("Features", numberOfClusters: 4));

// entrenamos el modelo
Console.WriteLine("Start training model....");
var model = pipeline.Fit(transformedData2);
Console.WriteLine("Model training complete!");

Start training model....
Model training complete!


In [38]:
// Obtener métricas de scoring del modelo
var predictions = model.Transform(transformedData2);
var metrics = mlContext.Clustering.Evaluate(predictions);

In [39]:
//Disponemos de varias métricas pero mostraremos el  average minimum score. 
// Nos dice la distancia media entre todas las muestras al centroide de su cluster. Por lo que cuanto más pequeño, mejor es el cluster

Console.WriteLine($"Average minimum score: {metrics.AverageDistance}");


Average minimum score: 65,26333216140107


In [40]:
//Persistimos el modelo a un fichero .ZIP
mlContext.Model.Save(model, transformedData2.Schema, "modelo.zip");
